# Open3D SLAM Pipeline
A notebook implementation of a calibrated SLAM (Simultaneous Localization and Mapping) pipeline using Open3D for point cloud registration and mapping.

## Import Required Libraries

In [2]:
import open3d as o3d
import numpy as np
import os
import re

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Define Helper Functions

In [7]:
# Function to sort filenames numerically (0, 1, 2, 10, 11...) 
# instead of lexicographically (0, 1, 10, 11, 2...)
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower()
            for text in re.split('([0-9]+)', s)]

## Define SLAM Pipeline Function

In [8]:
def run_calibrated_slam(data_folder):
    # 1. Load and Sort Files Correctly
    file_names = sorted([f for f in os.listdir(data_folder) if f.endswith('.pcd')], 
                        key=natural_sort_key)
    
    if not file_names:
        print(f"No PCD files found in {data_folder}")
        return

    print(f"Starting SLAM with {len(file_names)} frames in correct order...")

    # Initialize Global Map with the first frame
    global_map = o3d.io.read_point_cloud(os.path.join(data_folder, file_names[0]))
    
    # Pre-clean the initial map to remove sensor noise
    global_map, _ = global_map.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
    
    # Trajectory tracking
    current_pose = np.eye(4)
    trajectory_points = [current_pose[:3, 3]] 

    # 2. Iterative Registration Loop
    for i in range(1, len(file_names)):
        # Load source (new frame) and target (previous frame only)
        source = o3d.io.read_point_cloud(os.path.join(data_folder, file_names[i]))
        target = o3d.io.read_point_cloud(os.path.join(data_folder, file_names[i-1]))

        # Clean source data
        source, _ = source.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)

        # Pre-processing for ICP
        source_down = source.voxel_down_sample(voxel_size=0.05)
        target_down = target.voxel_down_sample(voxel_size=0.05)
        
        source_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
        target_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

        # 3. Compute FPFH features for global registration
        source_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
            source_down, o3d.geometry.KDTreeSearchParamHybrid(radius=0.25, max_nn=100))
        target_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
            target_down, o3d.geometry.KDTreeSearchParamHybrid(radius=0.25, max_nn=100))
        
        # 4. Global registration using RANSAC with FPFH features
        global_result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
            source_down, target_down, source_fpfh, target_fpfh,
            mutual_filter=True, max_correspondence_distance=0.2,
            estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPlane(),
            ransac_n=3, checkers=[], criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999)
        )
        
        # 5. Fine alignment with ICP using global registration result as initialization
        reg_result = o3d.pipelines.registration.registration_icp(
            source_down, target_down, max_correspondence_distance=0.1,
            init=global_result.transformation,
            estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPlane(
                o3d.pipelines.registration.HuberLoss(k=0.1)
            ),
            criteria=o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=100)
        )
        
        # 6. Accumulate pose transformation
        current_pose = current_pose @ reg_result.transformation
        trajectory_points.append(current_pose[:3, 3])
        
        # 7. Transform source by accumulated pose and merge with global map
        source.transform(current_pose)
        global_map += source

        # Print progress for every frame
        print(f"Frame {i} aligned successfully (fitness: {reg_result.fitness:.4f}).")

        # Periodic map cleaning to keep it sharp (less frequently)
        if i % 5 == 0:
            global_map = global_map.voxel_down_sample(voxel_size=0.01)

    # 5. Final Output
    global_map = global_map.voxel_down_sample(voxel_size=0.01)
    o3d.io.write_point_cloud("global_map.pcd", global_map)

    # 6. Visualization with Trajectory
    lines = [[i, i+1] for i in range(len(trajectory_points)-1)]
    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(trajectory_points),
        lines=o3d.utility.Vector2iVector(lines),
    )
    line_set.paint_uniform_color([1, 0, 0]) # Red trajectory line

    print("Opening 3D Classroom Visualization...")
    o3d.visualization.draw_geometries([global_map, line_set], 
                                      window_name="Corrected SLAM Map")

## Run the SLAM Pipeline
Ensure the 'data' folder contains your .pcd files before running this cell.

In [9]:
# Run the SLAM pipeline
run_calibrated_slam('data')

Starting SLAM with 20 frames in correct order...
Frame 1 aligned successfully (fitness: 0.9900).
Frame 2 aligned successfully (fitness: 0.9870).
Frame 3 aligned successfully (fitness: 0.9998).
Frame 4 aligned successfully (fitness: 0.9998).
Frame 5 aligned successfully (fitness: 0.9688).
Frame 6 aligned successfully (fitness: 0.9095).
Frame 7 aligned successfully (fitness: 0.8489).
Frame 8 aligned successfully (fitness: 0.8090).
Frame 9 aligned successfully (fitness: 0.7393).
Frame 10 aligned successfully (fitness: 0.9100).
Frame 11 aligned successfully (fitness: 0.7990).
Frame 12 aligned successfully (fitness: 0.8177).
Frame 13 aligned successfully (fitness: 0.9092).
Frame 14 aligned successfully (fitness: 0.9625).
Frame 15 aligned successfully (fitness: 0.9939).
Frame 16 aligned successfully (fitness: 0.9998).
Frame 17 aligned successfully (fitness: 0.9964).
Frame 18 aligned successfully (fitness: 1.0000).
Frame 19 aligned successfully (fitness: 0.9367).
Opening 3D Classroom Visualiz

## Visualize Saved Map (Without Re-aligning)
Load and display the previously generated global map without re-running the SLAM pipeline.


In [5]:
import os
import open3d as o3d
# Load and visualize the saved global map
if os.path.exists("global_map.pcd"):
    global_map = o3d.io.read_point_cloud("global_map.pcd")
    print(f"Loaded map with {len(global_map.points)} points")
    o3d.visualization.draw_geometries([global_map], window_name="SLAM Map Visualization")
else:
    print("global_map.pcd not found. Run the SLAM pipeline first.")


Loaded map with 285843 points
